In [ ]:
import cv2
#DeepLearningのために追加
from keras.models import load_model
import numpy as np
model=load_model("mnist.h5")

cap = cv2.VideoCapture(0)

while (True):
    #DeepLearningのために追加
    Xt=[]
    Yt=[]
    
    ret,frame = cap.read()
    
    #サイズでかいとき対策として半分にする
    h,w,_=frame.shape[:3]
    frame = cv2.resize(frame, (int(w/2), int(h/2)))
    
    h,w,_=frame.shape[:3]
    h_center=h//2
    w_center=w//2
    
    cv2.rectangle(frame,(w_center-71,h_center-71),(w_center+71,h_center+71),(255,0,0))
    
    im=frame[h_center-70:h_center+70,w_center-70:w_center+70]
    im=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    _,im =cv2.threshold(im,0,255,cv2.THRESH_OTSU)
    im=cv2.bitwise_not(im)
    im=cv2.GaussianBlur(im,(9,9),0)
    im=cv2.resize(im,(28,28),cv2.INTER_CUBIC)
    
    #DeepLearningのために追加
    Xt.append(im)
    Xt=np.array(Xt)/255
    Xt=np.expand_dims(Xt,axis=-1)
    result=model.predict(Xt,batch_size=1)
    
    #DeepLearningのために追加
    for i in range(10):
        r=round(result[0,i],2)
        r="{0:0.2f}".format(r)
        Yt.append([i,r]);
        Yt=sorted(Yt,key=lambda x:(x[1]))
    cv2.putText(frame,"1st:" + str(Yt[9]),(10,40),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),1,cv2.LINE_AA)
    
    
    cv2.startWindowThread()
    cv2.imshow("frame",frame)
    cv2.imshow("debug",im)
    key = cv2.waitKey(30) & 0xFF
    if key ==ord('q') :
        break
cap.release()
#フリーズ対策としてwaitKeyで挟む
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)